<a href="https://colab.research.google.com/github/Series-Parallel/UCR_Time_Series_Classification_Deep_Learning_From_Scratch/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lightning > /dev/null

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import lightning as L
from torch.optim import Adam

In [ ]:
def reducer(filename):
  data = np.loadtxt(filename, delimiter=',')
  Y = data[:,0]
  X = data[:,1:]
  return X, Y

In [ ]:
x_train, y_train = reducer("Adiac_TRAIN.txt")
x_test, y_test = reducer("Adiac_TEST.txt")

In [ ]:
classes = len(np.unique(y_test))

In [ ]:
y_train = ((y_train - y_train.min())/(y_train.max() - y_train.min()) * (classes - 1)).astype(int)
y_test = ((y_test - y_test.min())/ (y_test.max() - y_test.min()) * (classes - 1)).astype(int)

In [ ]:
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [ ]:
x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_train_mean) / x_train_std

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], 1, x_test.shape[1], 1)

In [ ]:
input_train = torch.tensor(x_train, dtype=torch.float32)
input_test = torch.tensor(x_test, dtype=torch.float32)

In [ ]:
train_dataset = TensorDataset(input_train, y_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [ ]:
class ResNetBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_sizes = (8, 5, 3)):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size= kernel_sizes[0], padding ='same')
    self.bn1 = nn.BatchNorm2d(out_channels)


    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size= kernel_sizes[1], padding ='same')
    self.bn2 = nn.BatchNorm2d(out_channels)

    self.conv3 = nn.Conv2d(out_channels, out_channels, kernel_size= kernel_sizes[2], padding ='same')
    self.bn3 = nn.BatchNorm2d(out_channels)

    self.shortcut = nn.Sequential()
    if in_channels != out_channels:
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_channels, out_channels, kernel_size=1, padding='same'),
          nn.BatchNorm2d(out_channels)
      )

  def forward(self, x):
    identity = self.shortcut(x)

    out = F.relu(self.bn1(self.conv1(x)))
    out = F.relu(self.bn2(self.conv2(out)))
    out = self.bn3(self.conv3(out))

    out += identity
    return F.relu(out)

In [ ]:
class ResNet(L.LightningModule):

  L.seed_everything(seed=813306)

  def __init__(self, input_shape, num_classes, n_feature_maps = 64):
    super().__init__()

    self.block1 = ResNetBlock(in_channels=1, out_channels=n_feature_maps)
    self.block2 = ResNetBlock(in_channels=n_feature_maps, out_channels=n_feature_maps*2)
    self.block3 = ResNetBlock(in_channels=n_feature_maps*2, out_channels=n_feature_maps*2)

    self.global_pool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(n_feature_maps*2, num_classes)
    self.loss = nn.CrossEntropyLoss()

  def forward(self,x):
    x = self.block1(x)
    x = self.block2(x)
    x = self.block3(x)

    self.last_conv_output = x.clone()

    x = self.global_pool(x)
    x = torch.flatten(x,1)
    x = self.fc(x)

    return x

  def training_step(self, batch, btach_idx):
    inputs, labels = batch
    outputs = self.forward(inputs)
    loss = self.loss(outputs,labels)
    self.log('train_loss', loss)
    return loss

  def configure_optimizers(self):
    return Adam(self.parameters(), lr=0.001)


In [ ]:
input_shape = input_train.shape[1:]

In [ ]:
num_classes = len(torch.unique(y_train_tensor))

In [ ]:
model = ResNet(input_shape, num_classes)

In [ ]:
trainer = L.Trainer(max_epochs=1500)
trainer.fit(model, train_dataloaders=train_dataloader)

In [ ]:
model.eval()

# Run predictions
with torch.no_grad():
    predictions = model(input_test)
    predicted_labels = torch.argmax(predictions, dim=1)
    accuracy = torch.sum(predicted_labels == y_test_tensor).item() / len(y_test_tensor)

print(f"Test Accuracy: {accuracy:.4f}")